<a href="https://colab.research.google.com/github/ogyogy/hello-ethereum/blob/main/hello_ethereum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ethereum入門

## 目的

Ethereumを理解するために、[web3.py](https://github.com/ethereum/web3.py) を実際に動かす。

## web3.pyのインストール

In [1]:
!pip install web3

今回はテスト用Ethereumノードを使用するため、追加の依存関係をインストールする。

In [2]:
!pip install "web3[tester]"

## アカウント間でETHを送金してみる

In [3]:
from web3 import Web3, EthereumTesterProvider

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


True

In [ ]:
# ローカルで動くテスト用のEthereumノードに接続
# このノードは自分だけのプライベートなネットワークで動くため
# トランザクションや残高の操作を安全に試せる
# パブリックなメインネットやテストネットに接続する場合は
# AlchemyやInfuraなどの外部サービスを利用する
w3 = Web3(EthereumTesterProvider())
w3.is_connected()

In [4]:
# 最新のブロックを取得
block = w3.eth.get_block('latest')
block

AttributeDict({'number': 0,
 'hash': HexBytes('0x66475af5f5b2121b3811d2c536dc122a3c21495dee469fd6c7942ecaf276ec94'),
 'parentHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00'),
 'transactionsRoot': HexBytes('0x56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421'),
 'receiptsRoot': HexBytes('0x56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421'),
 'stateRoot': HexBytes('0xf1588db9a9f1ed91effabdec31f93cb4212b008c8b8ba047fd55fabebf6fd727'),
 'miner': '0x0000000000000000000000000000000000000000',
 'difficulty': 0,
 'totalDifficulty': 0,
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'size': 616,
 'extraData': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 30029122,
 'gasUs

In [5]:
# ブロック番号を取得
# 取引が行われていないので0
block["number"]

0

In [6]:
# Ethereumアカウントの一覧を取得
# 表示される16進数の文字列はアカウントのアドレス
# 今回はテスト環境なので、最初から使えるテスト用アカウントが自動で用意されている
w3.eth.accounts

['0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 '0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69',
 '0x1efF47bc3a10a45D4B230B5d10E37751FE6AA718',
 '0xe1AB8145F7E55DC933d51a18c793F901A3A0b276',
 '0xE57bFE9F44b819898F47BF37E5AF72a0783e1141',
 '0xd41c057fd1c78805AAC12B0A94a405c0461A6FBb',
 '0xF1F6619B38A98d6De0800F1DefC0a6399eB6d30C',
 '0xF7Edc8FA1eCc32967F827C9043FcAe6ba73afA5c',
 '0x4CCeBa2d7D2B4fdcE4304d3e09a1fea9fbEb1528']

In [7]:
# 0番目のアカウントの残高をWei単位で取得
# Weiとは、Ethereumネットワーク上で使用される通貨であるEther(ETH)の最小単位
# 1 ETH = 1e18 Wei
# 浮動小数点数には計算誤差があるため、
# EthereumではETHを最小単位のWeiに分解し、整数として扱う設計になっている
addr0 = w3.eth.accounts[0]
balance0 = w3.eth.get_balance(addr0)
balance0

1000000000000000000000000

In [8]:
# 取得した0番目のアカウントの残高をETH単位に変換
# Ethereumは仮想通貨専用の仕組みではなく、
# 分散した計算基盤としてプログラムを実行するためのネットワーク
# ETHはその計算資源を利用するための支払い手段で、
# gasは実行した計算量を表す単位
w3.from_wei(balance0, "ether")

Decimal('1000000')

In [9]:
# トランザクションを実行
# アカウント0からアカウント1に1ETH送金
# 送金額とは別に、実行手数料(gas)は送信元アカウントが支払う
tx_hash = w3.eth.send_transaction({
    "from": w3.eth.accounts[0],
    "to": w3.eth.accounts[1],
    "value": w3.to_wei(1, "ether"),
})
tx_hash

HexBytes('0xf568762943fe39a82c25ae24206c640c445f07e105e4e31325518ca0a3b57ef3')

## トランザクション実行後のgas代を確認する

In [10]:
# 実行したトランザクションの内容を取得
# ここで表示されるgasに関する情報は、実際の使用量や支払額ではない
# gas: ガスリミット(この取引で使用してよいgas量の上限)
# gasPrice: 1gasあたりの価格(実際に適用された価格ではない)
tx = w3.eth.get_transaction(tx_hash)
tx

AttributeDict({'type': 2,
 'hash': HexBytes('0xf568762943fe39a82c25ae24206c640c445f07e105e4e31325518ca0a3b57ef3'),
 'nonce': 0,
 'blockHash': HexBytes('0xb0dd0f7f0d0e28ddb53faf236650eb71f724f48de80f0e38072756a67825f3b9'),
 'blockNumber': 1,
 'transactionIndex': 0,
 'from': '0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 'to': '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 'value': 1000000000000000000,
 'gas': 121000,
 'input': HexBytes('0x'),
 'chainId': 131277322940537,
 'accessList': [],
 'maxFeePerGas': 1000000000,
 'maxPriorityFeePerGas': 1000000000,
 'gasPrice': 1000000000,
 'v': 0,
 's': HexBytes('0x234ff6b9784566789952e16e8c7edfdc8d374d871ef65047e594b0a569053705'),
 'r': HexBytes('0xe34f58a9e92538dd31d0ad143b74cd185bc6ef3d5ebb3de1217131f91f78a205'),
 'y_parity': 0})

In [11]:
# トランザクションレシートを確認
receipt = w3.eth.get_transaction_receipt(tx_hash)
receipt

AttributeDict({'blockHash': HexBytes('0xb0dd0f7f0d0e28ddb53faf236650eb71f724f48de80f0e38072756a67825f3b9'),
 'blockNumber': 1,
 'contractAddress': None,
 'cumulativeGasUsed': 21000,
 'effectiveGasPrice': 1000000000,
 'from': '0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 'gasUsed': 21000,
 'logs': [],
 'state_root': b'\x01',
 'status': 1,
 'to': '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 'transactionHash': HexBytes('0xf568762943fe39a82c25ae24206c640c445f07e105e4e31325518ca0a3b57ef3'),
 'transactionIndex': 0,
 'type': 2})

In [12]:
# 実際に使用したgasを確認(Wei)
receipt["gasUsed"]

21000

In [13]:
# 実際に適用されたgasの最終単価を確認(Wei)
receipt["effectiveGasPrice"]

1000000000

In [14]:
# gas代を計算し、ETH単位で確認
gas_fee_wei = receipt["gasUsed"] * receipt["effectiveGasPrice"]
gas_fee_eth = w3.from_wei(gas_fee_wei, "ether")
gas_fee_eth

Decimal('0.000021')

In [15]:
# 0番目のアカウントの送金とgas支払い後の残高を取得し、ETH単位で確認
# 残高は送金額とgas代の分だけ減っている
addr0 = w3.eth.accounts[0]
balance0 = w3.eth.get_balance(addr0)
w3.from_wei(balance0, "ether")

Decimal('999998.999979')

In [16]:
# 1番目のアカウントの送金受け取り後の残高を取得し、ETH単位で確認
# 残高は送金額分(1ETH)だけ増えている
addr1 = w3.eth.accounts[1]
balance1 = w3.eth.get_balance(addr1)
w3.from_wei(balance1, "ether")

Decimal('1000001')

## トランザクション実行前にgas代を予測する

In [17]:
# このトランザクションを実行した場合に必要になる
# gas使用量を事前に見積もる
tx = {
    "from": w3.eth.accounts[0],
    "to": w3.eth.accounts[1],
    "value": w3.to_wei(1, "ether"),
}
estimated_gas = w3.eth.estimate_gas(tx)
estimated_gas

21000

In [18]:
# 現在のネットワーク状況をもとに、1gasあたりの単価を事前に見積もる
# EIP-1559では実際のgas単価は
# min(maxFeePerGas, baseFeePerGas + maxPriorityFeePerGas)
# で決まり、これにgas使用量を掛けたものがgas代
gas_price = w3.eth.gas_price
gas_price

1000000000

In [19]:
# gas代の合計を見積もる
estimated_fee_wei = estimated_gas * gas_price
w3.from_wei(estimated_fee_wei, "ether")

Decimal('0.000021')

In [20]:
# 最新のブロックを取得
block = w3.eth.get_block('latest')
block

AttributeDict({'number': 1,
 'hash': HexBytes('0xb0dd0f7f0d0e28ddb53faf236650eb71f724f48de80f0e38072756a67825f3b9'),
 'parentHash': HexBytes('0x66475af5f5b2121b3811d2c536dc122a3c21495dee469fd6c7942ecaf276ec94'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00'),
 'transactionsRoot': HexBytes('0x22b4575b5a132744e23dca5b63b5ab763fdf4dccfd63396d33ab57bdeb70f9dc'),
 'receiptsRoot': HexBytes('0xf78dfb743fbd92ade140711c8bbc542b5e307f0ab7984eff35d751969fe57efa'),
 'stateRoot': HexBytes('0x81e0d703bfa1a1456609416167c77a5a42fd81adae314bbe8dbcd75486ac627e'),
 'miner': '0x0000000000000000000000000000000000000000',
 'difficulty': 0,
 'totalDifficulty': 0,
 'mixHash': HexBytes('0x85380221cfec5ce282fc9615a09fafd8807328cec3db48a4e3bcc81ea86d82a2'),
 'size': 745,
 'extraData': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 30029122,
 'gasUs